# Mosaico 2021

In [1]:
import ee
import geemap
import geemap.chart as chart
import os
import folium

In [146]:
Map = geemap.Map()
Map2 = geemap.Map()
Map3 = geemap.Map()
Map4 = geemap.Map()

## Área de Estudio 

In [3]:
work_dir = os.path.expanduser('../../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)

## Funciones

In [4]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [5]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [6]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B3','B5']).rename('NDWI')
    return image.addBands(ndwi)

In [7]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B3','B4']).rename('NDGI')
    return image.addBands(ndgi)

In [8]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [9]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [10]:
def Cloud20(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(20)) 

In [11]:
def Cloud10(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(10)) 

In [12]:
def addCloud(image):
    nube = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.addBands(ee.Image(nube.select(['cloud'])).rename('nube'))

## Colecciones

### 2021

In [13]:
start_date = '2020-10-01'
end_date = '2021-03-31'

l8_2021 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [14]:
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [15]:
coleccion2021 = l8_2021.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate).map(addCloud)

In [16]:
NDVI_2021 = coleccion2021.qualityMosaic('NDVI')

In [17]:
coleccion_sin_nubes = coleccion2021.map(Cloud20)
greenest_222 = coleccion_sin_nubes.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021.add_ee_layer(greenest_222, vizParams, 'Mapa sin Nubes')
display(map_2021)

In [18]:
coleccion21 = coleccion2021.map(Cloud10)
greenest21 = coleccion21.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021.add_ee_layer(greenest21, vizParams, 'Mapa sin Nubes')
display(map_2021)

In [19]:
# coleccion21 = coleccion2021.map(Cloud10)
greenest21_v2 = coleccion2021.qualityMosaic('NDVI')
map_2021 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021.add_ee_layer(greenest21_v2, vizParams, 'Mapa sin Nubes')
display(map_2021)

In [20]:
coleccion21_ = coleccion2021.map(Cloud30)
agua21 = coleccion21_.qualityMosaic('NDWI')
map_2021_ = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021_.add_ee_layer(agua21, vizParams, 'Mapa sin Nubes')
display(map_2021_)

In [21]:
# coleccion21_ = coleccion2021.map(Cloud30)
agua21_v2 = coleccion2021.qualityMosaic('NDWI')
map_2021_ = folium.Map(location = [-45, -70], zoom_start = 4)
map_2021_.add_ee_layer(agua21_v2, vizParams, 'Mapa sin Nubes')
display(map_2021_)

In [22]:
Map.addLayer(agua21, vizParams, 'agua 2021')
Map.addLayer(coleccion2021, vizParams, 'coleccion 2021')
Map.addLayer(agua21_v2, vizParams, 'agua v2')
Map.addLayer(greenest21, vizParams, 'greenest 2021')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### 2020

In [122]:
start_date = '2019-10-01'
end_date = '2020-03-31'

l8_2020 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [127]:
coleccion2020 = l8_2020.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate).map(addCloud)

In [128]:
NDVI_2020 = coleccion2020.qualityMosaic('NDVI')

In [26]:
# coleccion20 = coleccion2020.map(Cloud10)
greenest20_v2 = coleccion2020.qualityMosaic('NDVI')
map_2020 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2020.add_ee_layer(greenest20_v2, vizParams, 'Mapa sin Nubes')
display(map_2020)

In [27]:
coleccion20 = coleccion2020.map(Cloud10)
greenest20 = coleccion20.qualityMosaic('NDVI')
map_2020 = folium.Map(location = [-45, -70], zoom_start = 4)
map_2020.add_ee_layer(greenest20, vizParams, 'Mapa sin Nubes')
display(map_2020)

In [45]:
coleccion20_ = coleccion2020.map(Cloud30)
agua20 = coleccion20_.qualityMosaic('NDWI')
map_2020_ = folium.Map(location = [-45, -70], zoom_start = 4)
map_2020_.add_ee_layer(agua20, vizParams, 'Mapa sin Nubes')
display(map_2020_)

### 2019

In [120]:
start_date = '2018-10-01'
end_date = '2019-03-31'

l8_2019 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [125]:
coleccion2019 = l8_2019.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate).map(addCloud)

In [126]:
NDVI_2019 = coleccion2019.qualityMosaic('NDVI')

In [129]:
green19 = coleccion2019.map(Cloud30)
green19 = green19.qualityMosaic('NDVI')
map_sur = folium.Map(location = [-45, -70], zoom_start = 4)
map_sur.add_ee_layer(green19, vizParams, 'Mapa sin Nubes')
display(map_sur)

### Año completo

In [29]:
start_date = '2018-10-01'
end_date = '2021-03-31'

l8_sur = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [30]:
coleccionsur = l8_sur.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate).map(addCloud)

In [31]:
greensur = coleccionsur.map(Cloud30)
greensur = greensur.qualityMosaic('NDVI')
map_sur = folium.Map(location = [-45, -70], zoom_start = 4)
map_sur.add_ee_layer(greensur, vizParams, 'Mapa sin Nubes')
display(map_sur)

In [53]:
muestra_sur = greensur.sample(patagonia_grande, 10000)

In [55]:
options = {"title": 'Fechas Composición',
          "xlabel": 'Date',
          "ylabel": 'Pixel count',
          "colors": ['#1BD4C3']
          }

In [56]:
property = 'date'

In [57]:
chart.feature_histogram(muestra_sur, property, maxBuckets=30, **options)

## Mosaico

In [32]:
nubes_agua = agua21_v2.select('nube') 
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map2.addLayer(nubes_agua, {'palette': palette}, 'nubes')

In [33]:
nubes21 = agua21.select('nube') 
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map2.addLayer(nubes21, {'palette': palette}, 'nubes21')

In [34]:
Map2

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [35]:
nubes_greenest = greenest21.select('nube') 
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map3.addLayer(nubes_greenest, {'palette': palette}, 'nubes1')

In [36]:
nubes_greenest21 = greenest21_v2.select('nube') 
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map3.addLayer(nubes_greenest21, {'palette': palette}, 'nubes2')

In [37]:
nubes_greenest20 = greenest20_v2.select('nube') 
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
Map3.addLayer(nubes_greenest20, {'palette': palette}, 'nubes3')

In [38]:
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [133]:
def llenado(image):
    return image.where(nubes_greenest21.gt(30),greenest20)

In [134]:
img1 = llenado(greenest21_v2)

In [135]:
map_img1 = folium.Map(location = [-45, -70], zoom_start = 4)
map_img1.add_ee_layer(img1, vizParams, 'Mapa sin Nubes')
display(map_img1)

In [73]:
# mosaico1 =  img1.set('SENSOR_ID', 'OLI_TIRS')


In [83]:
type(coleccion2021)

ee.imagecollection.ImageCollection

In [82]:
type(greenest21)

ee.image.Image

In [84]:
type(img1)

ee.image.Image

In [136]:
corregido = img1.set('SENSOR_ID', 'OLI_TIRS')

In [137]:
scored_mosaico= ee.Algorithms.Landsat.simpleCloudScore(corregido)
# img_nubes_mosaico = ee.Image(scored_mosaico)

In [138]:
type(scored_mosaico)

ee.image.Image

In [139]:
map_nube = folium.Map(location = [-45, -70], zoom_start = 4)
map_nube.add_ee_layer(scored_mosaico, vizParams , 'Mapa sin Nubes')
display(map_nube)

In [140]:
nubes_mosaico= scored_mosaico.select('cloud')
map_nube = folium.Map(location = [-45, -70], zoom_start = 4)
map_nube.add_ee_layer(nubes_mosaico, {'palette': palette} , 'Mapa sin Nubes')
display(map_nube)

In [58]:
# def llenado2(image):
#     return image.where(nubes_greenest21.eq(100),agua21)

In [59]:
# img2 = llenado2(img1) 

In [60]:
# map_img2 = folium.Map(location = [-45, -70], zoom_start = 4)
# map_img2.add_ee_layer(img2, vizParams, 'Mapa sin Nubes')
# display(map_img2)

In [141]:
def llenado3(image):
    return image.where(nubes_mosaico.gt(30),green19)

In [144]:
img2 = llenado3(img1) 

In [145]:
map_img2 = folium.Map(location = [-45, -70], zoom_start = 4)
map_img2.add_ee_layer(img2, vizParams, 'Mapa sin Nubes')
display(map_img2)

In [147]:
Map4.addLayer(greenest21_v2, vizParams, '0')
Map4.addLayer(img1, vizParams, '1')
Map4.addLayer(img2, vizParams, '2')
Map4

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…